# Database Design with SQLite
This project covers topics:
- Import data into SQLite
- Design a normalized database schema
- Create tables for our schema
- Insert data into our schema

We will be working with a file of [Major League Baseball](https://en.wikipedia.org/wiki/Major_League_Baseball) games from [Retrosheet](http://www.retrosheet.org/). Retrosheet compiles detailed statistics on baseball games from the 1800s through to today. The main file we will be working from **`game_log.csv`**, has been produced by combining 127 separate CSV files from retrosheet, and has been pre-cleaned to remove some inconsistencies. The game log has hundreds of data points on each game which we will normalize this data into several separate tables using SQL, providing a robust database of game-level statistics.

In addition to the main file, we have also included three 'helper' files, also sourced from Retrosheet:
- park_codes.csv
- person_codes.csv
- team_codes.csv

And additional file **`game_log_fields.txt`** included, which comprises colnames and their descriptions.

An important first step when working with any new data is to perform exploratory data analysis (EDA). EDA gets us familiar with the data and gives us a level of background knowledge that will help us throughout our project. The methods you use when performing EDA will depend on what you plan to do with the data. In our case, we're wanting to create a normalized database, so our focus should be:

- Becoming familiar, at a high level, with the meaning of each column in each file.
- Thinking about the relationships between columns within each file.
- Thinking about the relationships between columns across different files.

Let's get started.

## Imports

In [1]:
import pandas as pd
import sqlite3
pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)
!rm mlb.db

In [2]:
!cat game_log_fields.txt

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B" -- the second game of a double-header involving 3 teams
    3     Day of week  ("Sun","Mon","Tue","Wed","Thu","Fri","Sat")
  4-5     Visiting team and league
    6     Visiting team game number
          For this and the home team game number, ties are counted as
          games and suspended games are counted from the starting
          rather than the ending date.
  7-8     Home team and league
    9     Home team game number
10-11     Visiting and home team score (unquoted)


## EDA
Explore and get acquinted with the data

In [3]:
game_log_df = pd.read_csv('game_log.csv', dtype='str')
park_codes_df = pd.read_csv('park_codes.csv')
person_codes_df = pd.read_csv('person_codes.csv')
team_codes_df = pd.read_csv('team_codes.csv')

In [4]:
print('game_log shape: {}\npark_codes shape: {}\nperson_codes shape: {}\nteam_codes shape: {}'.format(game_log_df.shape,park_codes_df.shape,person_codes_df.shape,team_codes_df.shape))

game_log shape: (171907, 161)
park_codes shape: (252, 9)
person_codes shape: (20494, 7)
team_codes shape: (150, 8)


In [5]:
game_log_df.head(2)

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,NaN,1,FW1,NaN,1,0,2,54,D,NaN,NaN,NaN,FOR01,200,120,000000000,010010000,30,4,1,0,0,0,0,0,0,1,NaN,6,1,NaN,-1,NaN,4,1,1,1,0,0,27,9,0,3,0,0,31,4,1,0,0,2,0,0,0,1,NaN,0,0,NaN,-1,NaN,3,1,0,0,0,0,27,3,3,1,1,0,boakj901,John Boake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,NaN,NaN,NaN,NaN,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2,kimbg101,Gene Kimball,4,paboc101,Charlie Pabor,7,allia101,Art Allison,8,white104,Elmer White,9,prata101,Al Pratt,1,sutte101,Ezra Sutton,5,carlj102,Jim Carleton,3,bassj101,John Bass,6,selmf101,Frank Sellman,5,mathb101,Bobby Mathews,1,foraj101,Jim Foran,3,goldw101,Wally Goldsmith,6,lennb101,Bill Lennon,2,caret101,Tom Carey,4,mince101,Ed Mincher,7,mcdej101,James McDermott,8,kellb105,Bill Kelly,9,NaN,Y
1,18710505,0,Fri,BS1,NaN,1,WS3,NaN,1,20,18,54,D,NaN,NaN,NaN,WAS01,5000,145,107000435,640113030,41,13,1,2,0,13,0,0,0,18,NaN,5,3,NaN,-1,NaN,12,1,6,6,1,0,27,13,10,1,2,0,49,14,2,0,0,11,0,0,0,10,NaN,2,1,NaN,-1,NaN,14,1,7,7,0,0,27,20,10,2,3,0,dobsh901,Henry Dobson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,NaN,NaN,NaN,NaN,spala101,Al Spalding,braia102,Asa Brainard,wrigg101,George Wright,6,barnr102,Ross Barnes,4,birdd102,Dave Birdsall,9,mcvec101,Cal McVey,2,wrigh101,Harry Wright,8,goulc101,Charlie Gould,3,schah101,Harry Schafer,5,conef101,Fred Cone,7,spala101,Al Spalding,1,watef102,Fred Waterman,5,forcd101,Davy Force,6,mille105,Everett Mills,3,alli

In [6]:
game_log_df['v_league'].value_counts()

NL    88866
AL    74713
AA     5039
FL     1243
PL      532
UA      428
Name: v_league, dtype: int64

Game log file includes all possible info for a certain game, includes date, premises, teams, scores, game_lengths, specific game info.

Player defensive position designates a spot(base) of that player.

Leagues codes:
- AL - American League, constitutes one of the two Major Leagues
- NL - National League, constitutes one of the two Major Leagues
- AA - The American Association of Independent Professional Baseball is an independent professional baseball league founded in 2005
- FL - Florida Leagues, included in Minor League.
- PL - The Players' National League of Professional Base Ball Clubs, popularly known as the Players' League, was a short-lived but star-studded professional American baseball league of the 19th century.
- UA - The Union Association was a league in Major League Baseball which lasted for only one season in 1884

In [7]:
park_codes_df.head(2)

,park_id,name,aka,city,state,start,end,league,notes
0,ALB01,Riverside Park,NaN,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,NaN,Altoona,PA,04/30/1884,05/31/1884,UA,NaN


park_codes includes detailed info about the Park. It is related to the game log datafile by park_id column.

In [8]:
person_codes_df.head(2)

,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
0,aardd001,Aardsma,David,04/06/2004,NaN,NaN,NaN
1,aaroh101,Aaron,Hank,04/13/1954,NaN,NaN,NaN


A detailed info on the game participant, relates to the log datafile by id column.

In [9]:
team_codes_df.head(2)

,team_id,league,start,end,city,nickname,franch_id,seq
0,ALT,UA,1884,1884,Altoona,Mountain Cities,ALT,1
1,ARI,NL,1998,0,Arizona,Diamondbacks,ARI,1


A detailed info on the team, relates to the game log file by team_id column.

## Inserting data into database
First define functions for the ease of interaction with the connection object.


In [10]:
# A function, that takes a SQL query as an argument
# and returns a pandas dataframe of that query.
def run_query(query):
    with sqlite3.connect('mlb.db') as conn:
        return pd.read_sql(query,conn)
    
# A function that takes a SQL command as an argument and executes it using the sqlite module.
def run_command(query_command):
    with sqlite3.connect('mlb.db') as conn:
        conn.execute('PRAGMA foreign_keys = ON;') # Enables foreign key restraints
        conn.isolation_level = None
        conn.execute(query_command)
    
# A function that calls the run_query() function to return a list of all tables and views in the database.
def show_tables():
    query = 'SELECT name, type \
            FROM sqlite_master \
            WHERE type IN (\'table\',\'view\')'
    return run_query(query)

Create a connection to a new database and assign it to a variable object.

In [11]:
conn = sqlite3.connect('mlb.db')

Create unnormalized tables

In [12]:
%%time
game_log_df.to_sql('game_log', conn, index=False)

CPU times: user 10.9 s, sys: 276 ms, total: 11.2 s
Wall time: 11.4 s


In [13]:
park_codes_df.to_sql('park_codes', conn, index=False)
person_codes_df.to_sql('person_codes', conn, index=False)
team_codes_df.to_sql('team_codes', conn, index=False)

In [14]:
show_tables()

,name,type
0,game_log,table
1,park_codes,table
2,person_codes,table
3,team_codes,table


Great, we've created tables and have data imported.

Adding a **`game_id`** column with an id to form a 12-character id as following: `ATL198304080`. Where first 3 characters - team id, next are year, mo, day, and the last one is a game number.

Related columns in the game_log dataset would be:
 - h_name
 - date
 - number_of_game
 
 We should concatenate those to form a `game_id` column

In [95]:
q_show_top_rows = '''
    SELECT * FROM game_log
    WHERE protest IS NOT NULL
    LIMIT 5
    ;
'''

q_add_column = '''
    ALTER TABLE game_log
    ADD COLUMN game_id TEXT;
'''

q_populate_gameid = '''
    UPDATE OR IGNORE game_log
    SET game_id= h_name||date||number_of_game
    WHERE game_id is NULL;
'''

In [96]:
run_query(q_show_top_rows)

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info,game_id
0,19210528,0,Sat,CIN,NL,38,PIT,NL,35,3,4,53,D,"19210630,,3,2,45",None,Y,PIT06,None,121,000200010,000200011,31,6,0,1,1,3,1,0,0,4,1,1,0,1,0,0,6,1,3,3,0,0,26,14,3,0,2,0,28,7,1,0,1,4,3,0,1,1,0,2,0,1,0,0,3,2,3,3,0,0,27,9,0,0,0,0,brenb901,Bill Brennan,emslb101,Bob Emslie,None,None,None,None,None,None,None,None,morap101,Pat Moran,gibsg101,George Gibson,adamb104,Babe Adams,luqud101,Dolf Luque,None,None,grimc101,Charlie Grimm,luqud101,Dolf Luque,glazw101,Whitey Glazner,bohns101,Sam Bohne,5,daubj101,Jake Daubert,3,rouse101,Edd Roush,8,duncp101,Pat Duncan,7,kopfl101,Larry Kopf,6,fonsl101,Lew Fonseca,4,see-c101,Charlie See,9,wingi101,Ivey Wingo,2,luqud101,Dolf Luque,1,marar101,Rabbit Maranville,6,carem101,Max Carey,8,cutsg101,George Cutshaw,4,whitp102,Possum Whitted,9,tierc101,Cotton Tierney,5,grimc101,Charlie Grimm,3,mokaj101,Johnny Mokan,7,schmw102,Walter Schmidt,2,glazw101,Whitey Glazner,1,"umpchange,8,ump1b,klemb901",Y,PIT192105280
1,19240715,0,Tue,NY1,NL,80,CHN,NL,79,9,4,54,D,None,T,H,CHI11,None,117,001012500,000030100,36,10,2,0,2,8,2,0,0,4,None,1,0,0,None,0,6,2,3,0,0,0,27,10,1,0,2,0,31,7,1,0,1,3,0,0,0,3,None,4,0,1,None,0,3,3,7,0,0,0,27,9,1,0,0,0,klemb901,Bill Klem,wilsf901,Frank Wilson,None,None,None,None,None,None,None,None,mcgrj101,John McGraw,killb101,Bill Killefer,None,None,None,None,nehfa101,Art Nehf,None,None,deanw101,Wayland Dean,kauft101,Tony Kaufmann,younr101,Ross Youngs,9,frisf101,Frankie Frisch,4,soutb101,Billy Southworth,8,meusi101,Irish Meusel,7,kellg101,High Pockets Kelly,3,jackt101,Travis Jackson,6,grohh101,Heinie Groh,5,snydf101,Frank Sn

In [17]:
run_command(q_add_column)

In [18]:
run_command(q_populate_gameid)

In [97]:
run_query(q_show_top_rows)

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info,game_id
0,19210528,0,Sat,CIN,NL,38,PIT,NL,35,3,4,53,D,"19210630,,3,2,45",None,Y,PIT06,None,121,000200010,000200011,31,6,0,1,1,3,1,0,0,4,1,1,0,1,0,0,6,1,3,3,0,0,26,14,3,0,2,0,28,7,1,0,1,4,3,0,1,1,0,2,0,1,0,0,3,2,3,3,0,0,27,9,0,0,0,0,brenb901,Bill Brennan,emslb101,Bob Emslie,None,None,None,None,None,None,None,None,morap101,Pat Moran,gibsg101,George Gibson,adamb104,Babe Adams,luqud101,Dolf Luque,None,None,grimc101,Charlie Grimm,luqud101,Dolf Luque,glazw101,Whitey Glazner,bohns101,Sam Bohne,5,daubj101,Jake Daubert,3,rouse101,Edd Roush,8,duncp101,Pat Duncan,7,kopfl101,Larry Kopf,6,fonsl101,Lew Fonseca,4,see-c101,Charlie See,9,wingi101,Ivey Wingo,2,luqud101,Dolf Luque,1,marar101,Rabbit Maranville,6,carem101,Max Carey,8,cutsg101,George Cutshaw,4,whitp102,Possum Whitted,9,tierc101,Cotton Tierney,5,grimc101,Charlie Grimm,3,mokaj101,Johnny Mokan,7,schmw102,Walter Schmidt,2,glazw101,Whitey Glazner,1,"umpchange,8,ump1b,klemb901",Y,PIT192105280
1,19240715,0,Tue,NY1,NL,80,CHN,NL,79,9,4,54,D,None,T,H,CHI11,None,117,001012500,000030100,36,10,2,0,2,8,2,0,0,4,None,1,0,0,None,0,6,2,3,0,0,0,27,10,1,0,2,0,31,7,1,0,1,3,0,0,0,3,None,4,0,1,None,0,3,3,7,0,0,0,27,9,1,0,0,0,klemb901,Bill Klem,wilsf901,Frank Wilson,None,None,None,None,None,None,None,None,mcgrj101,John McGraw,killb101,Bill Killefer,None,None,None,None,nehfa101,Art Nehf,None,None,deanw101,Wayland Dean,kauft101,Tony Kaufmann,younr101,Ross Youngs,9,frisf101,Frankie Frisch,4,soutb101,Billy Southworth,8,meusi101,Irish Meusel,7,kellg101,High Pockets Kelly,3,jackt101,Travis Jackson,6,grohh101,Heinie Groh,5,snydf101,Frank Sn

In [20]:
q_find_saving_pitcher_meaning = '''
    SELECT DISTINCT h_name FROM game_log
    WHERE
        winning_pitcher_id='deand102'
        AND h_score>v_score
'''
run_query(q_find_saving_pitcher_meaning)

,h_name
0,SLN
1,NY1
2,BRO
3,PIT
4,CIN
5,PHI
6,BSN
7,CHN


Looks good. Our newly created column is shown as the last one. We've created uniq game identifier, that is compatible with Retrosheet [data dictionary](http://www.retrosheet.org/eventfile.htm).

## Data Normalization
The tables we will create are below, with some notes on the normalization choices made:

- `person`
    - Each of the 'debut' columns have been omitted, as the data will be able to be found from other tables.
    - Since the game log file has no data on coaches, we made the decision to not include this data.
- `park`
    - The start, end, and league columns contain data that is found in the main game log and can be removed.
- `league`
    - Because some of the older leagues are not well known, we will create a table to store league names.
- `appearance_type`
    - Our appearance table will include data on players with positions, umpires, managers, and awards (like winning pitcher). This table will store information on what different types of appearances are available.

Here is a schema for our baseball database:
![db_schema](data/images/db_design_sketch_dq.png)

## Tables creation

#### person table

We will populate `person` table based on data in `person_codes` table. Let's see first few rows of the source.

In [21]:
query_show_person_codes='''
    SELECT * FROM person_codes LIMIT 3;
'''
run_query(query_show_person_codes)

,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
0,aardd001,Aardsma,David,04/06/2004,None,None,None
1,aaroh101,Aaron,Hank,04/13/1954,None,None,None
2,aarot101,Aaron,Tommie,04/10/1962,None,04/06/1979,None


Create a `person` table, populating it, assigning proper datatypes.

In [22]:
command_create_person_t = '''
    CREATE TABLE IF NOT EXISTS person(
        person_id TEXT PRIMARY KEY,
        first_name TEXT,
        last_name TEXT
        );
'''
command_populate_person_t = '''
    INSERT OR IGNORE INTO person
    SELECT id, first, last FROM person_codes
    ;
'''
run_command(command_create_person_t)
run_command(command_populate_person_t)

Display first few rows of newly created table

In [23]:
query_show_person='''
    SELECT * FROM person LIMIT 3;
'''
run_query(query_show_person)

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron


#### park table

In [24]:
query_show_park_codes='''
    SELECT * FROM park_codes LIMIT 3;
'''
run_query(query_show_park_codes)

,park_id,name,aka,city,state,start,end,league,notes
0,ALB01,Riverside Park,None,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,04/30/1884,05/31/1884,UA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,04/19/1966,None,AL,None


Create a `park` table according to a schema, populating it, assigning proper datatypes.

In [25]:
command_create_park_t = '''
    CREATE TABLE IF NOT EXISTS park(
        park_id TEXT PRIMARY KEY,
        name TEXT,
        nickname TEXT,
        city TEXT,
        state TEXT,
        notes TEXT
        );
'''
command_populate_park_t = '''
    INSERT OR IGNORE INTO park
    SELECT park_id, name, aka, city, state, notes FROM park_codes
    ;
'''
run_command(command_create_park_t)
run_command(command_populate_park_t)

Display first few rows of newly created table

In [26]:
query_show_person='''
    SELECT * FROM park LIMIT 3;
'''
run_query(query_show_person)

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None


#### league table

Create a `league` table according to a schema, populating it manually, assigning proper datatypes.

In [27]:
command_create_league_t = '''
    CREATE TABLE IF NOT EXISTS league (
        league_id TEXT PRIMARY KEY,
        name TEXT
        );
'''
command_populate_league_t = '''
    INSERT OR IGNORE INTO league
    VALUES
        ("AL", "American League"),
        ("NL", "National League"),
        ("AA", "The American Association of Independent Professional Baseball"),
        ("FL", "Florida Leagues"),
        ("PL", "The Players' National League of Professional Base Ball Clubs"),
        ("UA", "The Union Association")
    ;
'''
run_command(command_create_league_t)
run_command(command_populate_league_t)

Display first few rows of newly created table

In [28]:
query_show_league='''
    SELECT * FROM league;
'''
run_query(query_show_league)

,league_id,name
0,AL,American League
1,NL,National League
2,AA,The American Association of Independent Professional Baseball
3,FL,Florida Leagues
4,PL,The Players' National League of Professional Base Ball Clubs
5,UA,The Union Association


#### appearance_type table
Data for table population is available in `appearance_type.csv`

In [29]:
appearance_type_df = pd.read_csv('appearance_type.csv')
appearance_type_df.head(5)

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense


In [110]:
comm_drop_apptype_t ='''
    DROP TABLE IF EXISTS appearance_type;
'''
comm_drop_apptype_source_t ='''
    DROP TABLE IF EXISTS appearance_type_source;
'''
run_command(comm_drop_apptype_t)
run_command(comm_drop_apptype_source_t)

In [111]:
# Create source table (with data)
with sqlite3.connect('mlb.db') as conn:
    appearance_type_df.to_sql('appearance_type_source', conn, index=False)

In [112]:
# Create a table for our database schema
comm_create_apptype_t = '''
    CREATE TABLE IF NOT EXISTS appearance_type
    (
    appearance_type_id TEXT PRIMARY KEY,
    name TEXT,
    category TEXT
    )
'''

run_command(comm_create_apptype_t)

In [116]:
# Populate appearance_type table with data from appearance_type_source table
comm_populate_appearance_type_t = '''
    INSERT OR IGNORE INTO appearance_type
    SELECT
        appearance_type_id,
        name,
        category
    FROM appearance_type_source
'''
run_command(comm_populate_appearance_type_t)

In [118]:
query_show_appearance_type_t ='''
    SELECT * FROM appearance_type;
'''
run_query(query_show_appearance_type_t)

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense
5,O6,Batter 6,offense
6,O7,Batter 7,offense
7,O8,Batter 8,offense
8,O9,Batter 9,offense
9,D1,Pitcher,defense


#### team table

In [32]:
command_create_team_t = '''
    CREATE TABLE IF NOT EXISTS team (
        team_id TEXT PRIMARY KEY,
        league_id TEXT,
        city TEXT,
        nickname TEXT,
        franch_id TEXT,
        FOREIGN KEY (league_id) REFERENCES league(league_id)
    );
'''
command_populate_team_t = '''
    INSERT OR IGNORE INTO team
    SELECT team_id, league, city, nickname, franch_id FROM team_codes;
'''

run_command(command_create_team_t)
run_command(command_populate_team_t)

In [33]:
query_show_team_rows = '''
    SELECT * FROM team LIMIT 5;
'''

run_query(query_show_team_rows)

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1


#### game table

In [34]:
command_create_game_t = '''
    CREATE TABLE IF NOT EXISTS game (
        game_id TEXT PRIMARY KEY,
        date TEXT,
        number_of_game INTEGER,
        park_id TEXT,
        length_outs TEXT,
        day BOOLEAN,
        completion TEXT,
        forfeit TEXT,
        protest TEXT,
        attendance INTEGER,
        length_minutes INTEGER,
        additional_info TEXT,
        acquisition_info TEXT,
        FOREIGN KEY (park_id) REFERENCES park(park_id)
    );
'''
command_populate_game_t = '''
    INSERT OR IGNORE INTO game
    SELECT 
        game_id, date, number_of_game, park_id, length_outs, day_night, completion,
        forefeit, protest, attendance, length_minutes, additional_info,
        acquisition_info
    FROM game_log
'''

run_command(command_create_game_t)
run_command(command_populate_game_t)

In [35]:
query_show_game_rows = '''
    SELECT * FROM game LIMIT 5;
'''
run_query(query_show_game_rows)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forfeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,FW1187105040,18710504,0,FOR01,54,D,None,None,None,200,120,None,Y
1,WS3187105050,18710505,0,WAS01,54,D,None,None,None,5000,145,HTBF,Y
2,RC1187105060,18710506,0,RCK01,54,D,None,None,None,1000,140,None,Y
3,CH1187105080,18710508,0,CHI01,54,D,None,None,None,5000,150,None,Y
4,TRO187105090,18710509,0,TRO01,54,D,None,None,None,3250,145,HTBF,Y


#### team_appearance table

The table is huge with lots of columns to populate. Rather than type in colnames manually, let's get it from the game_log table schema.

In [36]:
q_game_log_schema = '''
    SELECT sql FROM sqlite_master
    WHERE name = "game_log"
          AND type = "table";
'''
comm_game_log_schema = '''
    PRAGMA table_info(game_log)
'''

In [37]:
log_schema_df = run_query(q_game_log_schema)
log_schema_df

sql
0  CREATE TABLE "game_log" (\n"date" TEXT,\n  "number_of_game" TEXT,\n  "day_of_week" TEXT,\n  "v_name" TEXT,\n  "v_league" TEXT,\n  "v_game_number" TEXT,\n  "h_name" TEXT,\n  "h_league" TEXT,\n  "h_game_number" TEXT,\n  "v_score" TEXT,\n  "h_score" TEXT,\n  "length_outs" TEXT,\n  "day_night" TEXT,\n  "completion" TEXT,\n  "forefeit" TEXT,\n  "protest" TEXT,\n  "park_id" TEXT,\n  "attendance" TEXT,\n  "length_minutes" TEXT,\n  "v_line_score" TEXT,\n  "h_line_score" TEXT,\n  "v_at_bats" TEXT,\n  "v_hits" TEXT,\n  "v_doubles" TEXT,\n  "v_triples" TEXT,\n  "v_homeruns" TEXT,\n  "v_rbi" TEXT,\n  "v_sacrifice_hits" TEXT,\n  "v_sacrifice_flies" TEXT,\n  "v_hit_by_pitch" TEXT,\n  "v_walks" TEXT,\n  "v_intentional_walks" TEXT,\n  "v_strikeouts" TEXT,\n  "v_stolen_bases" TEXT,\n  "v_caught_stealing" TEXT,\n  "v_grounded_into_double" TEXT,\n  "v_first_catcher_interference" TEXT,\n  "v_left_on_base" TEXT,\n  "v_pitchers_used" TEXT,\n  "v_individual_earned_runs" TEXT,\n  "v_team_earned_runs" TEXT,\n  "v_wild_pitches" TEXT,\n  "v_balks" TEXT,\n  "v_putouts" TEXT,\n  "v_assists" TEXT,\n  "v_errors" TEXT,\n  "v_passed_balls" TEXT,\n  "v_double_plays" TEXT,\n  "v_triple_plays" TEXT,\n  "h_at_bats" TEXT,\n  "h_hits" TEXT,\n  "h_doubles" TEXT,\n  "h_triples" TEXT,\n  "h_homeruns" TEXT,\n  "h_rbi" TEXT,\n  "h_sacrifice_hits" TEXT,\n  "h_sacrifice_flies" TEXT,\n  "h_hit_by_pitch" TEXT,\n  "h_walks" TEXT,\n  "h_intentional_walks" TEXT,\n  "h_strikeouts" TEXT,\n  "h_stolen_bases" TEXT,\n  "h_caught_stealing" TEXT,\n  "h_grounded_into_double" TEXT,\n  "h_first_catcher_interference" TEXT,\n  "h_left_on_base" TEXT,\n  "h_pitchers_used" TEXT,\n  "h_individual_earned_runs" TEXT,\n  "h_team_earned_runs" TEXT,\n  "h_wild_pitches" TEXT,\n  "h_balks" TEXT,\n  "h_putouts" TEXT,\n  "h_assists" TEXT,\n  "h_errors" TEXT,\n  "h_passed_balls" TEXT,\n  "h_double_plays" TEXT,\n  "h_triple_plays" TEXT,\n  "hp_umpire_id" TEXT,\n  "hp_umpire_name" TEXT,\n  "1b_umpire_id" TEXT,\n  "1b_umpire_name" TEXT,\n  "2b_umpire_id" TEXT,\n  "2b_umpire_name" TEXT,\n  "3b_umpire_id" TEXT,\n  "3b_umpire_name" TEXT,\n  "lf_umpire_id" TEXT,\n  "lf_umpire_name" TEXT,\n  "rf_umpire_id" TEXT,\n  "rf_umpire_name" TEXT,\n  "v_manager_id" TEXT,\n  "v_manager_name" TEXT,\n  "h_manager_id" TEXT,\n  "h_manager_name" TEXT,\n  "winning_pitcher_id" TEXT,\n  "winning_pitcher_name" TEXT,\n  "losing_pitcher_id" TEXT,\n  "losing_pitcher_name" TEXT,\n  "saving_pitcher_id" TEXT,\n  "saving_pitcher_name" TEXT,\n  "winning_rbi_batter_id" TEXT,\n  "winning_rbi_batter_id_name" TEXT,\n  "v_starting_pitcher_id" TEXT,\n  "v_starting_pitcher_name" TEXT,\n  "h_starting_pitcher_id" TEXT,\n  "h_starting_pitcher_name" TEXT,\n  "v_player_1_id" TEXT,\n  "v_player_1_name" TEXT,\n  "v_player_1_def_pos" TEXT,\n  "v_player_2_id" TEXT,\n  "v_player_2_name" TEXT,\n  "v_player_2_def_pos" TEXT,\n  "v_player_3_id" TEXT,\n  "v_player_3_name" TEXT,\n  "v_player_3_def_pos" TEXT,\n  "v_player_4_id" TEXT,\n  "v_player_4_name" TEXT,\n  "v_player_4_def_pos" TEXT,\n  "v_player_5_id" TEXT,\n  "v_player_5_name" TEXT,\n  "v_player_5_def_pos" TEXT,\n  "v_player_6_id" TEXT,\n  "v_player_6_name" TEXT,\n  "v_player_6_def_pos" TEXT,\n  "v_player_7_id" TEXT,\n  "v_player_7_name" TEXT,\n  "v_player_7_def_pos" TEXT,\n  "v_player_8_id" TEXT,\n  "v_player_8_name" TEXT,\n  "v_player_8_def_pos" TEXT,\n  "v_player_9_id" TEXT,\n  "v_player_9_name" TEXT,\n  "v_player_9_def_pos" TEXT,\n  "h_player_1_id" TEXT,\n  "h_player_1_name" TEXT,\n  "h_player_1_def_pos" TEXT,\n  "h_player_2_id" TEXT,\n  "h_player_2_name" TEXT,\n  "h_player_2_def_pos" TEXT,\n  "h_player_3_id" TEXT,\n  "h_player_3_name" TEXT,\n  "h_player_3_def_pos" TEXT,\n  "h_player_4_id" TEXT,\n  "h_player_4_name" TEXT,\n  "h_player_4_def_pos" TEXT,\n  "h_player_5_id" TEXT,\n  "h_player_5_name" TEXT,\n  "h_player_5_def_pos" TEXT,\n  "h_player_6_id" TEXT,\n  "h_player_6_name" TEXT,\n  "h_player_6_def_pos" TEXT,\n  "h_player_7_id" TEXT,\n  "h_player_7_name" TEXT,\n  "h_player_7_def_pos" TEXT,

UGH! Looks messy! Let's get colnames and types relevant for our `team_appearance` table for home and visiting team separately.
We can leverage this by the means of regular expressions!

In [38]:
import re
colnames = log_schema_df.iloc[0,0]
home_team_stats_l = re.findall(r'\"(h_\w+)\" (TEXT)',colnames)
home_team_colnames = [column_details_tuple[0] for column_details_tuple in home_team_stats_l if not any(exclude_words in column_details_tuple[0] for exclude_words in ['player','manager','starting_pitcher','game_number'])]
home_team_colnames_type = [' '.join(column_details_tuple) for column_details_tuple in home_team_stats_l if not any(exclude_words in column_details_tuple[0] for exclude_words in ['player','manager','starting_pitcher','game_number'])]

In [39]:
re.sub(r'h_','' ,', '.join(home_team_colnames_type))

'name TEXT, league TEXT, score TEXT, line_score TEXT, at_bats TEXT, hits TEXT, doubles TEXT, triples TEXT, homeruns TEXT, rbi TEXT, sacrifice_hits TEXT, sacrifice_flies TEXT, hit_by_pitch TEXT, walks TEXT, intentional_walks TEXT, strikeouts TEXT, stolen_bases TEXT, caught_stealing TEXT, grounded_into_double TEXT, first_catcher_interference TEXT, left_on_base TEXT, pitchers_used TEXT, individual_earned_runs TEXT, team_earned_runs TEXT, wild_pitches TEXT, balks TEXT, putouts TEXT, assists TEXT, errors TEXT, passed_balls TEXT, double_plays TEXT, triple_plays TEXT'

In [40]:
home_team_quering = ', '.join(home_team_colnames)
home_team_quering

'h_name, h_league, h_score, h_line_score, h_at_bats, h_hits, h_doubles, h_triples, h_homeruns, h_rbi, h_sacrifice_hits, h_sacrifice_flies, h_hit_by_pitch, h_walks, h_intentional_walks, h_strikeouts, h_stolen_bases, h_caught_stealing, h_grounded_into_double, h_first_catcher_interference, h_left_on_base, h_pitchers_used, h_individual_earned_runs, h_team_earned_runs, h_wild_pitches, h_balks, h_putouts, h_assists, h_errors, h_passed_balls, h_double_plays, h_triple_plays'

In [41]:
visiting_team_quering = re.sub(r'h_','v_',home_team_quering)
visiting_team_quering

'v_name, v_league, v_score, v_line_score, v_at_bats, v_hits, v_doubles, v_triples, v_homeruns, v_rbi, v_sacrifice_hits, v_sacrifice_flies, v_hit_by_pitch, v_walks, v_intentional_walks, v_strikeouts, v_stolen_bases, v_caught_stealing, v_grounded_into_double, v_first_catcher_interference, v_left_on_base, v_pitchers_used, v_individual_earned_runs, v_team_earned_runs, v_wild_pitches, v_balks, v_putouts, v_assists, v_errors, v_passed_balls, v_double_plays, v_triple_plays'

Good, we have colnames and type as 1 string, which we can copy paste it into the query that creates table, and for a query which fetches data from the game_log table.

_When creating a table_, some colnames to edit manually:
- colname to write down manually: `game_id`
- column to rename manually: `name` to `team_id`, `league` to `league_id`

In [43]:
comm_create_team_app = '''
    CREATE TABLE IF NOT EXISTS team_appearance
        (
        game_id TEXT,
        team_id TEXT,
        home BOOLEAN,
        league_id TEXT,
        score TEXT, line_score TEXT, at_bats TEXT, hits TEXT, doubles TEXT,
        triples TEXT, homeruns TEXT, rbi TEXT, sacrifice_hits TEXT,
        sacrifice_flies TEXT, hit_by_pitch TEXT, walks TEXT,
        intentional_walks TEXT, strikeouts TEXT, stolen_bases TEXT,
        caught_stealing TEXT, grounded_into_double TEXT,
        first_catcher_interference TEXT, left_on_base TEXT, pitchers_used TEXT,
        individual_earned_runs TEXT, team_earned_runs TEXT, wild_pitches TEXT,
        balks TEXT, putouts TEXT, assists TEXT, errors TEXT, passed_balls TEXT,
        double_plays TEXT, triple_plays TEXT,
        
        PRIMARY KEY (team_id, game_id),
        FOREIGN KEY (team_id) REFERENCES team(team_id),
        FOREIGN KEY (game_id) REFERENCES game(game_id),
        FOREIGN KEY (league_id) REFERENCES league(league_id)
        );
'''

query_populate_team_app_t = '''
    INSERT OR IGNORE INTO team_appearance
        SELECT
            game_id,
            h_name,
            1 AS home,
            h_league, h_score, h_line_score, h_at_bats, h_hits, h_doubles,
            h_triples, h_homeruns, h_rbi, h_sacrifice_hits, h_sacrifice_flies,
            h_hit_by_pitch, h_walks, h_intentional_walks, h_strikeouts,
            h_stolen_bases, h_caught_stealing, h_grounded_into_double,
            h_first_catcher_interference, h_left_on_base, h_pitchers_used,
            h_individual_earned_runs, h_team_earned_runs, h_wild_pitches,
            h_balks, h_putouts, h_assists, h_errors, h_passed_balls,
            h_double_plays, h_triple_plays
        FROM game_log
        
    UNION
    
        SELECT
            game_id,
            v_name,
            0 AS home,
            v_league, v_score, v_line_score, v_at_bats, v_hits, v_doubles,
            v_triples, v_homeruns, v_rbi, v_sacrifice_hits, v_sacrifice_flies,
            v_hit_by_pitch, v_walks, v_intentional_walks, v_strikeouts,
            v_stolen_bases, v_caught_stealing, v_grounded_into_double,
            v_first_catcher_interference, v_left_on_base, v_pitchers_used,
            v_individual_earned_runs, v_team_earned_runs, v_wild_pitches,
            v_balks, v_putouts, v_assists, v_errors, v_passed_balls,
            v_double_plays, v_triple_plays
        FROM game_log;
'''

run_command(comm_create_team_app)
run_command(query_populate_team_app_t)

In [44]:
q_show_team_app_rows = '''
    SELECT * FROM team_appearance LIMIT 5;
'''
run_query(q_show_team_app_rows)

,game_id,team_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,ALT188404300,ALT,1,UA,2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,ALT188404300,SLU,0,UA,15,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,ALT188405020,ALT,1,UA,3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,ALT188405020,SLU,0,UA,16,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,ALT188405030,ALT,1,UA,5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


We have our large table `team_appearance` ready and steady.

#### person_appearance table

In [53]:
comm_create_personapp_table = '''
    CREATE TABLE IF NOT EXISTS person_appearance
    (
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id TEXT,
    
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
    );
'''

In [87]:
comm_populate_nonplayer_persons = '''
    INSERT INTO person_appearance (
        game_id,
        team_id,
        person_id,
        appearance_type_id
    )
        SELECT
            game_id,
            NULL,
            lf_umpire_id,
            "ULF"
        FROM game_log
        WHERE lf_umpire_id IS NOT NULL

    UNION

        SELECT
            game_id,
            NULL,
            rf_umpire_id,
            "URF"
        FROM game_log
        WHERE rf_umpire_id IS NOT NULL
        
    UNION

        SELECT
            game_id,
            NULL,
            hp_umpire_id,
            "UHP"
        FROM game_log
        WHERE hp_umpire_id IS NOT NULL
        
    UNION

        SELECT
            game_id,
            NULL,
            [1b_umpire_id],
            "U1B"
        FROM game_log
        WHERE [1b_umpire_id] IS NOT NULL
        
    UNION

        SELECT
            game_id,
            NULL,
            [2b_umpire_id],
            "U2B"
        FROM game_log
        WHERE [2b_umpire_id] IS NOT NULL
        
    UNION

        SELECT
            game_id,
            NULL,
            [3b_umpire_id],
            "U3B"
        FROM game_log
        WHERE [3b_umpire_id] IS NOT NULL

    UNION

        SELECT
            game_id,
            v_name,
            v_manager_id,
            "MM"
        FROM game_log
        WHERE v_manager_id IS NOT NULL

    UNION

        SELECT
            game_id,
            h_name,
            h_manager_id,
            "MM"
        FROM game_log
        WHERE h_manager_id IS NOT NULL

    UNION

        SELECT
            game_id,
            CASE
                WHEN h_score > v_score THEN h_name
                ELSE v_name
                END,
            winning_pitcher_id,
            "AWP"
        FROM game_log
        WHERE winning_pitcher_id IS NOT NULL

    UNION
    
        SELECT
            game_id,
            CASE
                WHEN h_score < v_score THEN h_name
                ELSE v_name
                END,
            losing_pitcher_id,
            "ALP"
        FROM game_log
        WHERE losing_pitcher_id IS NOT NULL
        
    UNION
    
        SELECT
            game_id,
            CASE
                WHEN h_score > v_score THEN h_name
                ELSE v_name
                END,
            saving_pitcher_id,
            "ASP"
        FROM game_log
        WHERE saving_pitcher_id IS NOT NULL
        
    UNION
    
        SELECT
            game_id,
            CASE
                WHEN h_score > v_score THEN h_name
                ELSE v_name
                END,
            winning_rbi_batter_id,
            "AWB"
        FROM game_log
        WHERE winning_rbi_batter_id IS NOT NULL
        
    UNION

        SELECT
            game_id,
            h_name,
            h_starting_pitcher_id,
            "PSP"
        FROM game_log
        WHERE h_starting_pitcher_id IS NOT NULL
        
    UNION

        SELECT
            game_id,
            v_name,
            v_starting_pitcher_id,
            "PSP"
        FROM game_log
        WHERE v_starting_pitcher_id IS NOT NULL;
'''

In [49]:
template_populate_players = """
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
"""

In [120]:
q_delete_personapp_t = '''
    DROP TABLE IF EXISTS person_appearance;
'''
run_command(q_delete_personapp_t)

In [121]:
run_command(q_delete_personapp_t)

# Create table
run_command(comm_create_personapp_table)

# Populate with persons (non-players)
run_command(comm_populate_nonplayer_persons)

# Populate with players persons
for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        # run commmand is a helper function which runs
        # a query against our database.
        run_command(template_populate_players.format(**query_vars))

In [127]:
q_show_personapp_rows = '''
    SELECT * FROM person_appearance LIMIT 5;
'''
run_query(q_show_personapp_rows)

,appearance_id,person_id,team_id,game_id,appearance_type_id
0,1,mccob104,None,CIN191908030,ULF
1,2,hurle901,None,NYA194910010,ULF
2,3,honoj901,None,NYA194910020,ULF
3,4,sudoe901,None,BRO195708201,ULF
4,5,sudoe901,None,BRO195708220,ULF


Hooray! A very complex table has been populated.

### House cleaning
Remove unnecessary tables

In [134]:
show_tables()

,name,type
0,person,table
1,park,table
2,league,table
3,team,table
4,game,table
5,team_appearance,table
6,appearance_type,table
7,person_appearance,table


In [133]:
comm_drop_t1 = '''
    DROP TABLE IF EXISTS game_log;
'''
comm_drop_t2 = '''
    DROP TABLE IF EXISTS park_codes;
'''
comm_drop_t3 = '''
    DROP TABLE IF EXISTS team_codes;
'''
comm_drop_t4 = '''
    DROP TABLE IF EXISTS person_codes;
'''
comm_drop_t5 = '''
    DROP TABLE IF EXISTS appearance_type_source;
'''
run_command(comm_drop_t1)
run_command(comm_drop_t2)
run_command(comm_drop_t3)
run_command(comm_drop_t4)
run_command(comm_drop_t5)

In [134]:
show_tables()

,name,type
0,person,table
1,park,table
2,league,table
3,team,table
4,game,table
5,team_appearance,table
6,appearance_type,table
7,person_appearance,table


That's it!

## Conclusion

This project covered:

- Import CSV data into a database.
- Design a normalized schema for a large, predominantly single table data set.
- Create tables that match the schema design.
- Migrate data from unnormalized tables into our normalized tables.